### AutoDiff Demos

The following notebook contains a walkthrough demonstration of how to use this automatic differentiation package. If you have not already, please be sure to install the package using pip:

In [ ]:
!pip install auto_diff_pkg

Start by importing the AutoDiff package (make sure to have it properly installed it first!)

In [ ]:
#import auto_diff_pkg.AutoDiff as AutoDiff
import auto_diff_pkg.AutoDiff as AD

# import the reverse mode package separately
import auto_diff_pkg.ReverseAutoDiff as RM

Now we can create an AutoDiff object and see what's included.

In [ ]:
ad = AD.AutoDiff(5.0) # auto differentiation object with value of 5
print(ad.__dir__()) 

The AutoDiff object has two primary class variables: its value (val) and its derivative (der, which by default is 1). There are custom dunder method implementations to carry out both value and derivative calculations. For example, adding a scalar to an AutoDiff object:

In [ ]:
ad1 = AD.AutoDiff(5.0) 
ad2 = ad + 5.0
print('value: {}'.format(ad2.val))
print('derivative: {}'.format(ad2.der))

Unsurprisingly, 5 + 5 is equal to 10, but the derivative of a scalar addition is 0, so the initial derivative of our AutoDiff object remains unchanged.

Adding two AutoDiff objects does change the derivative of their sum, however.

In [ ]:
ad1 = AD.AutoDiff(5.0)
ad2 = AD.AutoDiff(3.0)
ad3 = ad1 + ad2

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

More operations than addition can be applied to an AutoDiff object. Here are some of the following:

In [ ]:
ad1 = AD.AutoDiff(5.0)
ad2 = AD.AutoDiff(3.0)

print('---multiplication---')
ad3 = ad1 * ad2

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

print('\n---powers---')
ad3 = ad1 ** ad2

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

print('\n---sine---')
ad3 = AD.sin(ad1)

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

Please refer to the documentation for a full list of the operations supported by AutoDiff objects.

AutoDiff objects can also be used as input to lambda functions, increasing their usability and flexibility:

In [ ]:
f = lambda x: 2.0 * AD.sin(x)**2 + 3.0

ad = AD.AutoDiff(5.0)
ad2 = f(ad)

print('value: {}'.format(ad2.val))
print('derivative: {}'.format(ad2.der))

### Newton's method

Below we demonstrate how our AutoDiff implementation can be used to calculate Newton's method for finding equation roots.
This method converges very quickly usually and is very useful. 

In [ ]:
def newton_root_finder(f,guess, num_iter=15, epsilon=10**-10):
    x0 = AD.AutoDiff(guess)
    for n in range(0,num_iter):
        curr_iter = f(x0)
        if abs(curr_iter.val) < epsilon:
            print('Root found at ',x0.val,' after ', n, 'iterations.')
            return x0.val
        der = curr_iter.der
        if der == 0:
            print('Derivative is 0. Cannot solve')
            return None
        x0 = x0 - curr_iter.val/curr_iter.der
    print('Could not converge in requested number of iterations')
    return None

In [ ]:
f = lambda x: 2*(x**3) - x - 4
newton_root_finder(f,1)

### Multiple Functions and Variables

AutoDiff objects are also able to handle multiple functions and multiple variables. 

In [ ]:
x0=AD.AutoDiff(1,1,3,0) #value 1, derivative 1, 3 variables in total, this one is position 0
x1=AD.AutoDiff(2,1,3,1) #value 2, derivative 1, 3 variables in total, this one is position 1
x2=AD.AutoDiff(4,1,3,2) #value 4, derivative 1, 3 variables in total, this one is position 2

f1 = x0 + x1 + x2
f2 = 1*x0 + 2*x1 + 3*x2
f3 = 1*x0*2*x1*3*x2
f4 = x0**2 + x1**3 + x2**4
print(f4.der, "\n")

F=[f1, f2, f3, f4]
print(F[0].der)
print(F[1].der)
print(F[2].der)
print(F[3].der)

The same can be performed using the built-in jacobian function:

In [ ]:
#function takes multiple functions and assigns parameters from list. outputs jacobian
values = [1,2,4] 
def f1(x0, x1, x2):
    return (x0 + x1 + x2)
def f2(x0, x1, x2):
    return (1*x0 + 2*x1 + 3*x2)
def f3(x0, x1, x2):
    return (1*x0*2*x1*3*x2)
def f4(x0, x1, x2):
    return (x0**2 + x1**3 + x2**4)
functions = [f1, f2, f3,f4]
print(AD.jacobian(values, functions))

### Reverse Mode

This auto-differentiation package also handles the reverse mode of automatic differentiation. The following shows how a reverse mode object tracks its trace, which can be used to solve the complete graph

In [ ]:
x0 = RM.ReverseADNode(2.0)
x1 = RM.ReverseADNode(4.0)
x2 = RM.ReverseADNode(6.0)

f = x0 * x1 + (x2/2)

print(f.value)
x0.graph()

The following demonstration constructs the jacobian from 4 predefined functions of 3 variables using the built-in jacobian function for reverse mode objects.

In [ ]:
#function takes multiple functions and assigns parameters from list. outputs jacobian
values = [1,2,4] 
def f1(x0, x1, x2):
    return (x0 + x1 + x2)
def f2(x0, x1, x2):
    return (1*x0 + 2*x1 + 3*x2)
def f3(x0, x1, x2):
    return (1*x0*2*x1*3*x2)
def f4(x0, x1, x2):
    return (x0**2 + x1**3 + x2**4)
functions = [f1, f2, f3,f4]
print(RM.jacobian(values, functions))